# Initialisation

> Setting up the device to synchronise systems

In [ ]:
#| default_exp device

In [ ]:
#| hide
from nbdev.showdoc import *

# Import required libraries
import serial
import serial.tools.list_ports
import time
import numpy as np

The software drivers provide a simple interface to communicate with the device in order to send event signals from behavioural tasks to synchronise with recorded data.

## Importing the drivers
Once installed, the drivers can be imported into the task script like any package.

In [ ]:
# Import driver package
import wavewriter

All contents of the driver package can then be accessed using the prefix `syncmaster.`, i.e. the `SyncMaster` object is accessed using `syncmaster.SyncMaster()`.

Alternatively, all contents of the package can be imported directly into the present script's namespace:

In [ ]:
# Import all package contents into current namespace
from wavewriter import *

Package contents can then be accessed using their names directly without the need for a prefix, i.e. `SyncMaster()`.

While slightly more convenient, this runs the risk of colliding with function and object names imported from other packages, so we recommend using the package prefix where possible.

## Creating the device object
The software communicates with the device hardware using a software object. All device commands are controlled using this object.

In [ ]:
#| export
'''
    WaveWriter device drivers

    Oxford Neural Interfacing
    Written by Conor Keogh
    conor.keogh@nds.ox.ac.uk
    04/03/2024

    Provides functions for interacting with WaveWriter device
'''

'''
WaveWriter device class
Provide functions to interact with device
Finds device and confirms presence
Sends messages to device to specify waveforms and start/stop stimulation
'''
import serial
import serial.tools.list_ports
import time
import numpy as np

class WaveWriter:

    def __init__(self):
        '''
        Device object for controlling synchronisation
        '''

        '''
        Define constants for device configuration
        '''
        # Define messages
        self.GREETING = b'Hello'
        self.RESPONSE = 'Hi there'
        
        self.prep1_command = 'prep1'    # Prepare buffer 1 (V)
        self.done1_command = 'done1'    # Stop buffer 1
        self.start_command = 'start'    # Start stimulation
        self.stop_command = 'stop'      # Stop stimulation
    
        # Define COM port settings
        self.BAUDRATE = 115200
        
        # Flag for whether device connected
        self.connected = False
        
        # Empty arrays for waveform
        self.v = np.array([])
        
    def connect(self):
        # Get all serial ports
        ports = serial.tools.list_ports.comports()

        # For each port: try accessing and checking for acknowledge message
        port_found = False
        for port in ports:
            print(port)
            try:
                # Connect to serial port
                self.ser = serial.Serial(port.device, self.BAUDRATE, timeout=1, write_timeout=1)

                # Send test message and read response; repeat 3 times and keep third
                for _ in range(3):
                    self.ser.write(self.GREETING)
                    response = self.ser.readline()
                    print(response)

                # Check if response is appropriate
                if response.decode().rstrip('\x00') == self.RESPONSE:
                    self.target_port = port.device
                    port_found = True

                # Close port
                self.ser.close()

            except Exception as e:
                # Do nothing - just ignore failed portsA
                print(e)
                pass

        # If port found: connect to port
        if port_found:
            self.ser = serial.Serial(self.target_port, self.BAUDRATE, timeout=5)
            self.connected = True
            
        # If port not found: raise error
        else:
            raise Exception("Device not found")
            #print("Device not found")

    ''' Send required messages over serial '''
    # Send message via serial port
    def sendMessage(self, message):
        '''
        Send message over serial port
        Takes message to send
        '''
        if self.connected == True:
            self.ser.write(message.encode())
            
        # If not connected: raise error
        else:
            raise Exception("Device not connected - use WaveWriter.connect()")
        
    # Check inputs are appropriate
    def check_inputs(self, v):
        # Check types
        if type(v) is not np.ndarray:
            raise Exception("V is not an array")
            
        # Check dimensions
        v = np.squeeze(v)
        
        if v.ndim > 1:
            raise Exception("V is not one-dimensional")
            
        # If appropriate: save V to object
        self.v = v
        
    def convert_buffer(self, x):
        ''' Convert array to buffer to send '''
        # Create emoty string
        x_buffer = ''
        
        # Iterate through array
        for value in x[:-1]:
            x_buffer += str(value)    # Add to buffer
            x_buffer += ','           # Add delimiter
        x_buffer += str(x[-1])        # Add last value without delimiter
        
        return x_buffer
            
    def send_waveform(self, v):
        ''' Send waveform data to device '''
        # Check inputs are appropriate
        self.check_inputs(v)
        
        # Convert to string buffers to send
        v_buffer = self.convert_buffer(v)
        
        # Prepare to send first buffer
        self.sendMessage(self.prep1_command)
        time.sleep(1)
        
        # Send V to buffer 1
        self.sendMessage(v_buffer)
        time.sleep(1)
        
        # End buffer
        self.sendMessage(self.done1_command)
        time.sleep(1)
        
    def start(self):
        ''' Send start signal '''
        self.sendMessage(self.start_command)

    def stop(self):
        ''' Send end signal '''
        self.sendMessage(self.stop_command)

    # Close channel
    def close(self):
        '''
        Closes device connection
        '''
        self.ser.close()
        self.connected = False

In order to use the device in a task script, we must first create the device object. This should be done once at the beginning of the script.

In [ ]:
#|eval: false
# Create device object
device = wavewriter.WaveWriter()

This automatically carries out all initialisation procedures, including locating the device on the host system and ensuring the device is communicating correctly.

Note that an error will be produced on attempting to create the device object if the device is not connected to the host system.

Once the device has been initialised once in this way, it is ready to send event signals to the recording system as outlined in the `triggering` section.

## Shutting down the device
On completing the task, the communicating channel between the host system and the device should be shut down.

In [ ]:
show_doc(WaveWriter.close)

The device should be shut down using the `close` command at the end of the task. 

In [ ]:
#|eval: false
# Close device
device.close()

It can then be disconnected from the host system.

This ensures that all communications ports are closed correctly to avoid any errors.

::: {.dark-mode}
![Oxford Neural Interfacing 2023](oni.png)
:::

::: {.light-mode}
![Oxford Neural Interfacing 2023](oni_blue.png)
:::

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()